In [2]:
import pandas as pd
df = pd.read_csv('../data/yelp_review_polarity/train.csv',names=['rating','review'])
df['rating'] = df['rating'] - 1
df = df[['review','rating']]
df

,review,rating
0,"Unfortunately, the frustration of being Dr. Go...",0
1,Been going to Dr. Goldberg for over 10 years. ...,1
2,I don't know what Dr. Goldberg was like before...,0
3,I'm writing this review to give you a heads up...,0
4,All the food is great here. But the best thing...,1
...,...,...
559995,Ryan was as good as everyone on yelp has claim...,1
559996,Professional \nFriendly\nOn time AND affordabl...,1
559997,Phone calls always go to voicemail and message...,0
559998,Looks like all of the good reviews have gone t...,0


In [13]:
pos = df[df['rating']==1]
pos

,review,rating
1,Been going to Dr. Goldberg for over 10 years. ...,1
4,All the food is great here. But the best thing...,1
8,Before I finally made it over to this range I ...,1
9,I drove by yesterday to get a sneak peak. It ...,1
11,Wonderful reuben. Map shown on Yelp page is i...,1
...,...,...
559989,When Corey sent the email about tickets being ...,1
559991,"We tried the Cheese Danish w lemon. Creamy, fl...",1
559995,Ryan was as good as everyone on yelp has claim...,1
559996,Professional \nFriendly\nOn time AND affordabl...,1


In [14]:
neg = df[df['rating']!=1]
neg

,review,rating
0,"Unfortunately, the frustration of being Dr. Go...",0
2,I don't know what Dr. Goldberg was like before...,0
3,I'm writing this review to give you a heads up...,0
5,Wing sauce is like water. Pretty much a lot of...,0
6,Owning a driving range inside the city limits ...,0
...,...,...
559992,"Long line, inefficient staff. Maybe my expecta...",0
559993,First: Please note that many people are review...,0
559994,I'm shocked everyone time I go to a Bouchon Ba...,0
559997,Phone calls always go to voicemail and message...,0


In [15]:
cutPoint = int(round(0.8 * pos.shape[0]))

In [16]:
train = pd.concat([pos[0:cutPoint],neg[0:cutPoint]])
train.shape

(448000, 2)

In [17]:
val = pd.concat([pos[cutPoint:],neg[cutPoint:]])
val.shape

(112000, 2)

In [18]:
test = pd.read_csv('../data/yelp_review_polarity/test.csv',names=['rating','review'])
test['rating'] = test['rating'] - 1
test

,rating,review
0,1,"Contrary to other reviews, I have zero complai..."
1,0,Last summer I had an appointment to get new ti...
2,1,"Friendly staff, same starbucks fair you get an..."
3,0,The food is good. Unfortunately the service is...
4,1,Even when we didn't have a car Filene's Baseme...
...,...,...
37995,0,If I could give 0...I would. Don't do it.
37996,1,Items Selected:\nChocolate Cinnamon Horn\nSmal...
37997,0,Expensive lunch meals. Fried pickles were goo...
37998,0,Highly overpriced and food was cold. Our waitr...


In [19]:
train.to_csv('../data/yelp_review/train.csv',encoding='utf_8_sig',index=False)
val.to_csv('../data/yelp_review/val.csv',encoding='utf_8_sig',index=False)
test.to_csv('../data/yelp_review/test.csv',encoding='utf_8_sig',index=False)

# Test for T-E-GRU

In [2]:
import pandas as pd
df = pd.read_csv('../data/yelp_review/train.csv')
df


,review,rating
0,Been going to Dr. Goldberg for over 10 years. ...,1
1,All the food is great here. But the best thing...,1
2,Before I finally made it over to this range I ...,1
3,I drove by yesterday to get a sneak peak. It ...,1
4,Wonderful reuben. Map shown on Yelp page is i...,1
...,...,...
447995,"Ew, this location was crawling with ankle bite...",0
447996,TIPS FOR NEXT YEAR...\nMAKE SURE TIMES ARE COR...,0
447997,This used to be in Tivoli but has been moved o...,0
447998,This was my first time attending the festival....,0


In [ ]:
from nltk import tokenize
import re
import gensim
word2Vec_Model = gensim.models.KeyedVectors.load_word2vec_format( '../models/embeddings/glove-wiki-gigaword-300.gz')


In [ ]:
import os
from tqdm.auto import tqdm

sumOfTokens = 0
sumOutOfNum = 0
tokensList = []
for text in tqdm(df['review']):
    text = re.sub("<br />|\'s|\'", "", text)
    tokens = tokenize.word_tokenize(text)
    outOfNum = 0
    for word in tokens:
        try:
            word2Vec_Model.vocab[word].index
        except:
            outOfNum += 1
    sumOfTokens += len(tokens)
    sumOutOfNum += outOfNum
    tokensList.append((len(tokens),outOfNum))

In [6]:
1-sumOutOfNum/sumOfTokens

0.881274175916536

In [7]:
df[['tokensNum','outOfNum']] = pd.DataFrame(tokensList)

In [11]:
df[df['tokensNum']<500].shape[0] / df.shape[0]

0.9699464285714285

# Test for BERT

In [6]:
df

,review,rating
0,Been going to Dr. Goldberg for over 10 years. ...,1
1,All the food is great here. But the best thing...,1
2,Before I finally made it over to this range I ...,1
3,I drove by yesterday to get a sneak peak. It ...,1
4,Wonderful reuben. Map shown on Yelp page is i...,1
...,...,...
447995,"Ew, this location was crawling with ankle bite...",0
447996,TIPS FOR NEXT YEAR...\nMAKE SURE TIMES ARE COR...,0
447997,This used to be in Tivoli but has been moved o...,0
447998,This was my first time attending the festival....,0


In [7]:
from transformers import AutoTokenizer
Pre_TrainPath = "../models/bert-base-cased/"

tokenizer = AutoTokenizer.from_pretrained(Pre_TrainPath)

In [5]:
import re
def getTokenslen(text):
    text = re.sub("<br />|\'s|\'", "", text)
    tokens = tokenizer(text, return_tensors='pt',return_length=True,return_token_type_ids=False,return_attention_mask=False)
    return tokens['length'].item(),(tokens['input_ids']==100).sum().item()
getTokenslen('hello world !')

(5, 0)

In [40]:
from tqdm.auto import tqdm
sumOfTokens = 0
outOfNum = 0
tokensLength = []
for text in tqdm(df['review']):
    tempLen = getTokenslen(text)
    tokensLength.append(tempLen)
    sumOfTokens += tempLen[0]
    outOfNum += tempLen[1] 

  0%|          | 0/448000 [00:00<?, ?it/s]

In [41]:
1-outOfNum/sumOfTokens

0.9999998347026975

In [42]:
df[['NumOfTokens','outOfNum']] = pd.DataFrame(tokensLength)

In [45]:
df[df['NumOfTokens']<510].shape[0] / df.shape[0]

0.9656785714285714

In [13]:
import pandas as pd
import re
df = pd.read_csv('../data/yelp_review/test.csv')

In [14]:
def cleanText(text):
    text = re.sub("<br />|\'s|\'", "", text)
    return text
data = df
data['review'] = data['review'].map(cleanText)
labels = data['rating'].tolist()
tokens = tokenizer(data['review'].tolist(), return_tensors='pt', max_length=510, padding='max_length',truncation=True)
data_db = [(tokens['input_ids'][i], tokens['token_type_ids'][i], tokens['attention_mask'][i], labels[i]) for i in range(len(labels))]

In [17]:
len(tokens[1])

510

In [16]:
len(data['review'].tolist())

38000